# Overview
This file contains an example workflow to create a customized TIPP reference package, using some example genes included under `example_data/` with their necessary files:
1. the alignment,
2. the taxid of sequences,
3. the mapping between each sequence and its taxid.

----------------------
# Required Packages
```
taxtastic>=0.10.0
dendropy>=4.5.2
```

In [1]:
import os, sys
from subprocess import Popen, PIPE, STDOUT

# number of threads to use for raxml-ng and fasttree-2
num_threads = 4

# initialize work directory
workdir = 'example_data'

# where to write the TIPP reference package to
refpkg_dir = 'custom_tipp_refpkg'

# initialize the list of genes to use, which are assumed to be under the working directory
genes = ['RplA_COG0081', 'RplB_COG0090', 'RplD_COG0088', 'RpsC_COG0092', 'RpsM_COG0099']

# Needed files
In this example, we are only using one gene to create our customized TIPP reference package, included under `example_data/RplO_COG0200`. Remember that there are 4 files that we need to prepare (one for global use, and three for each gene):
1. `ncbi_taxonomy.db  - (global) NCBI taxonomy file created with taxtastic`
2. `est.aln.nuc.fasta[.gz] - (each gene) alignment file (can be gzipped)`
3. `species.txt       - (each gene) taxids of sequences in alignment, in the same order as in the alignment`
4. `species.mapping   - (each gene) mapping between sequences and taxids, in the same order as in the alignment`

The three files for the example gene are included, and the NCBI taxonomy database can be built below.

# Step 0: NCBI taxonomy database - `ncbi_taxonomy.db`

In [2]:
# download the latest NCBI taxdmp.zip
url = "https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdmp.zip"

# run taxtastic to create a database locally
if not os.path.exists(f"{workdir}/ncbi_taxonomy.db"):
    cmd = f"wget -P {workdir} {url}"
    os.system(cmd)

    cmd = f"taxit new_database -z {workdir}/taxdmp.zip"
    os.system(cmd)

    # move the created ncbi_taxonomy.db file to workdir
    os.system(f"mv ncbi_taxonomy.db {workdir}/")

# Step 1: Update species taxids

In [3]:
db_path = os.path.join(workdir, 'ncbi_taxonomy.db')

# two files to update
old2new_map = {'species.txt': 'species.updated.txt',
               'species.mapping': 'species.updated.mapping',
              }

for gene in genes:
    indir = os.path.join(workdir, gene)

    for infile, outfile in old2new_map.items():
        inpath = os.path.join(indir, infile)
        outpath = os.path.join(indir, outfile)
        # run taxit update_taxids
        cmd = f"taxit update_taxids {inpath} {db_path} -o {outpath}"
        os.system(cmd)

# Step 2: Get taxonomy table from the included species

In [4]:
db_path = os.path.join(workdir, 'ncbi_taxonomy.db')

for gene in genes:
    indir = os.path.join(workdir, gene)
    
    # get taxonomy.table with taxit
    species_path = os.path.join(indir, 'species.updated.txt')
    taxtable_path = os.path.join(indir, 'taxonomy.table')
    cmd = f"taxit taxtable {db_path} -i {species_path} -o {taxtable_path}"
    os.system(cmd)
    
    # get a cleaned version of the taxonomy table by removing redundant columns
    alltaxon_path = os.path.join(indir, 'all_taxon.taxonomy')
    cmd = f"python clean_taxonomy_table.py {taxtable_path} {alltaxon_path}"
    os.system(cmd)

# Step 3: Initialize taxonomy tree

In [5]:
for gene in genes:
    print('>>>>', gene)
    indir = os.path.join(workdir, gene)

    # run script build_taxonomic_tree.py
    # NOTE: both species.updated.txt and species.updated.mapping may be updated
    #       if there are unifurication detected in the taxonomy tree.
    #       e.g., if we have the following taxids: 57493 --> 1275 --> 136273
    #       then, both nodes 1275 and 136273 have sequences assigne to them.
    #       However, since node 1275 only has one child, it will be "contracted" when
    #       eliminating unifurication. Thus, sequences assigned to it may not appear
    #       in the unrefined.taxonomy.renamed file and leads to error when running RAxML-ng
    #
    #       This is a rare event, and the current solution is to modify nodes so that if we find
    #       a unifuricating node with taxid and sequences are assigned to it, we will contract all
    #       branches below this node and re-assign the sequences belonging to leaves of this subtree
    #       (rooted at the unifuricating node) to this node.
    #       Then, both species.updated.txt and species.updated.mapping would be updated to reflect
    #       the reassignment of taxids (to the sequences that are affected).
    species_path = os.path.join(indir, 'species.updated.txt')
    mapping_path = os.path.join(indir, 'species.updated.mapping')
    alltaxon_path = os.path.join(indir, 'all_taxon.taxonomy')
    unrefined_path = os.path.join(indir, 'unrefined.taxonomy')
    cmd = f"python build_taxonomic_tree.py {alltaxon_path} {species_path} {mapping_path} {unrefined_path}"
    p = Popen(cmd, stdout=PIPE, stderr=STDOUT, text=True, shell=True)
    for line in p.stdout:
        print(line.replace('\n', ''))
    
    # rename the taxonomy tree with sequence names from this gene
    # with script build_unrefined_tree.pl
    taxonomy_path = os.path.join(indir, 'unrefined.taxonomy.renamed')
    cmd = f"perl build_unrefined_tree.pl {mapping_path} {unrefined_path} {taxonomy_path}"
    os.system(cmd)

>>>> RplA_COG0081
Child taxids at root (131567): 2,2157
>>>> RplB_COG0090
Child taxids at root (131567): 2
	Re-assigning sequences with taxid 136273 to 1275
Reassigning taxids to sequences, backing up example_data/RplB_COG0090/species.updated.txt and example_data/RplB_COG0090/species.updated.mapping ...
GCF_001483755.1_NZ_LQBK01000002.1_cds_WP_058872530.1_238 taxid reassigned from 136273 to 1275
Reassigned taxids for 1/5510 sequences
>>>> RplD_COG0088
Child taxids at root (131567): 2
	Re-assigning sequences with taxid 1178541 to 561879
	Re-assigning sequences with taxid 444177 to 1421
Reassigning taxids to sequences, backing up example_data/RplD_COG0088/species.updated.txt and example_data/RplD_COG0088/species.updated.mapping ...
GCF_000017965.1_NC_010382.1_cds_WP_012296062.1_4284 taxid reassigned from 444177 to 1421
GCF_000691165.1_NZ_ASJD01000012.1_cds_WP_024425598.1_3451 taxid reassigned from 1178541 to 561879
Reassigned taxids for 2/5462 sequences
>>>> RpsC_COG0092
Child taxids at 

# (for tipp3) Get RAxML-ng maximum likelihood tree and model

In [6]:
for gene in genes:
    indir = os.path.join(workdir, gene)
    
    # masked the alignment first with >= 95% gaps
    aln_path = os.path.join(indir, 'est.aln.nuc.fasta.gz') 
    masked_path = os.path.join(indir, 'est.aln.nuc.masked95.fasta')
    thres = 0.95
    cmd = f"python mask_alignment.py {aln_path} {masked_path} {thres}"
    os.system(cmd)

In [7]:
for gene in genes:
    # run raxml-ng
    masked_path = os.path.join(indir, 'est.aln.nuc.masked95.fasta')
    check_path = os.path.join(indir, 'est.raxml.bestTree')
    if not os.path.exists(check_path):
        # resolve polytomies in the taxonomy tree and remove internal node labels to run RAxML-ng with
        taxonomy_path = os.path.join(indir, 'unrefined.taxonomy.renamed')
        starting_tree_path = os.path.join(indir, 'raxml.starting.tre')
        cmd = f"python resolve_polytomies.py {taxonomy_path} {starting_tree_path}"
        os.system(cmd)

        # Run RAxML-ng with the starting tree and the original taxonomy tree as the constraint
        cmd = ['raxml-ng', '--msa', masked_path, '--model GTR+G',
               '--tree', starting_tree_path, '--tree-constraint', taxonomy_path,
               '--brlen scaled',
               '--force perf_threads', '--redo',
               f'--prefix {indir}/est',
               '--threads', str(num_threads)]

        p = Popen(' '.join(cmd), stdout=PIPE, stderr=STDOUT, text=True, shell=True)
        for line in p.stdout:
            if not line.startswith('WARNING'):
                print(line.replace('\n', ''))

In [8]:
# add back the taxonomy information to the raxml output tree, and reroot the tree at taxid 131567
for gene in genes:
    print('>>>>', gene)
    indir = os.path.join(workdir, gene)
    
    # reroot the raxml-ng bestTree to 131567 (cellular organism) and relabel the tree
    root = 131567
    best_tree_path = os.path.join(indir, 'est.raxml.bestTree')
    taxonomy_path = os.path.join(indir, 'unrefined.taxonomy.renamed')
    rooted_tree_path = os.path.join(indir, 'est.raxml.bestTree.rooted')
    cmd = f"python relabel_tree.py {taxonomy_path} {best_tree_path} {rooted_tree_path} {root}"
    ret = os.popen(cmd).read()
    print(ret)

>>>> RplA_COG0081
reseeding example_data/RplA_COG0081/est.raxml.bestTree
relabeling example_data/RplA_COG0081/est.raxml.bestTree
	mapped 637
	missing 0
sanity check correctness True

>>>> RplB_COG0090
reseeding example_data/RplB_COG0090/est.raxml.bestTree (only one superkingdom group)
relabeling example_data/RplB_COG0090/est.raxml.bestTree
	mapped 653
	missing 0
sanity check correctness True

>>>> RplD_COG0088
reseeding example_data/RplD_COG0088/est.raxml.bestTree (only one superkingdom group)
relabeling example_data/RplD_COG0088/est.raxml.bestTree
	mapped 635
	missing 0
sanity check correctness True

>>>> RpsC_COG0092
reseeding example_data/RpsC_COG0092/est.raxml.bestTree
relabeling example_data/RpsC_COG0092/est.raxml.bestTree
	mapped 641
	missing 0
sanity check correctness True

>>>> RpsM_COG0099
reseeding example_data/RpsM_COG0099/est.raxml.bestTree (only one superkingdom group)
relabeling example_data/RpsM_COG0099/est.raxml.bestTree
	mapped 664
	missing 0
sanity check correctness T

# (for tipp3-accurate) Get FastTree-2 maximum likelihood tree and log

In [9]:
for gene in genes:
    print('>>>>', gene)
    indir = os.path.join(workdir, gene)
    
    # use the original alignment and raxml-ng tree to re-estimate FastTree-2 numeric parameters
    rooted_tree_path = os.path.join(indir, 'est.raxml.bestTree.rooted')
    alignment_path = os.path.join(indir, 'est.aln.nuc.fasta.gz')
    ft_log_path = os.path.join(indir, 'est.fasttree.log')
    ft_tree_path = os.path.join(indir, 'est.fasttree.tre')
    
    # enable multi-thread FastTree-2 (using binary FastTreeMP)
    os.system(f"export OMP_NUM_THREADS={num_threads}")
    
    # nucleotide alignment
    cmd = ['FastTreeMP', '-nosupport', '-gtr', '-gamma', '-nt']
    cmd.extend([
        '-mllen', '-nome',
        '-log', ft_log_path, '-intree', rooted_tree_path,
    ])
    
    if alignment_path.split('.')[-1] == 'gz':
        cmd = ['gzip', '-d', alignment_path, '-c', '|'] + cmd
    else:
        cmd = ['cat', alignment_path, '|'] + cmd
    out_fptr = open(ft_tree_path, 'w')
    p = Popen(' '.join(cmd), stdout=out_fptr, stderr=PIPE, shell=True, text=True)#, stdin=in_fptr)
    for line in p.stderr:
        print(line.replace('\n', ''))
    out_fptr.close()

>>>> RplA_COG0081
FastTree Version 2.1.11 SSE3, OpenMP (12 threads)
Alignment: standard input
Nucleotide distances: Jukes-Cantor Joins: balanced Support: none
Start at tree from example_data/RplA_COG0081/est.raxml.bestTree.rooted (no NNI) (no SPR)
ML Model: Generalized Time-Reversible, CAT approximation with 20 rate categories
Total branch-length 174.238 after 0.13 sec
      0.14 seconds: ML Lengths 1 of 1897 splits
      0.26 seconds: ML Lengths 401 of 1897 splits
      0.38 seconds: ML Lengths 801 of 1897 splits
      0.51 seconds: ML Lengths 1201 of 1897 splits
      0.63 seconds: ML Lengths 1601 of 1897 splits
1 rounds ML lengths: LogLk = -540809.088 Max-change 0.2599 Time 0.74
      0.74 seconds: Optimizing GTR model, step 1 of 12
      1.28 seconds: Optimizing GTR model, step 2 of 12
      1.66 seconds: Optimizing GTR model, step 3 of 12
      2.03 seconds: Optimizing GTR model, step 4 of 12
      2.49 seconds: Optimizing GTR model, step 5 of 12
      3.07 seconds: Optimizing GTR

      9.54 seconds: ML Lengths 201 of 1878 splits
      9.65 seconds: ML Lengths 401 of 1878 splits
      9.75 seconds: ML Lengths 601 of 1878 splits
      9.86 seconds: ML Lengths 801 of 1878 splits
      9.97 seconds: ML Lengths 1001 of 1878 splits
     10.08 seconds: ML Lengths 1201 of 1878 splits
     10.19 seconds: ML Lengths 1401 of 1878 splits
     10.30 seconds: ML Lengths 1601 of 1878 splits
     10.41 seconds: ML Lengths 1801 of 1878 splits
2 rounds ML lengths: LogLk = -530784.133 Max-change 0.4836 Time 10.47
     10.51 seconds: ML Lengths 101 of 1878 splits
     10.62 seconds: ML Lengths 301 of 1878 splits
     10.77 seconds: ML Lengths 601 of 1878 splits
     10.87 seconds: ML Lengths 801 of 1878 splits
     10.98 seconds: ML Lengths 1001 of 1878 splits
     11.08 seconds: ML Lengths 1201 of 1878 splits
     11.18 seconds: ML Lengths 1401 of 1878 splits
     11.29 seconds: ML Lengths 1601 of 1878 splits
     11.39 seconds: ML Lengths 1801 of 1878 splits
3 rounds ML lengths:

3 rounds ML lengths: LogLk = -455753.396 Max-change 0.0667 Time 11.66
     11.65 seconds: ML Lengths 1 of 1788 splits
     11.80 seconds: ML Lengths 201 of 1788 splits
     11.94 seconds: ML Lengths 401 of 1788 splits
     12.09 seconds: ML Lengths 601 of 1788 splits
     12.24 seconds: ML Lengths 801 of 1788 splits
     12.39 seconds: ML Lengths 1001 of 1788 splits
     12.53 seconds: ML Lengths 1201 of 1788 splits
     12.67 seconds: ML Lengths 1401 of 1788 splits
     12.82 seconds: ML Lengths 1601 of 1788 splits
4 rounds ML lengths: LogLk = -455752.875 Max-change 0.0182 Time 12.98
     12.97 seconds: ML Lengths 1 of 1788 splits
     13.12 seconds: ML Lengths 201 of 1788 splits
     13.26 seconds: ML Lengths 401 of 1788 splits
     13.41 seconds: ML Lengths 601 of 1788 splits
     13.56 seconds: ML Lengths 801 of 1788 splits
     13.70 seconds: ML Lengths 1001 of 1788 splits
     13.84 seconds: ML Lengths 1201 of 1788 splits
     13.99 seconds: ML Lengths 1401 of 1788 splits
     14

     16.29 seconds: Site likelihoods with rate category 10 of 20
     16.43 seconds: Site likelihoods with rate category 12 of 20
     16.58 seconds: Site likelihoods with rate category 14 of 20
     16.72 seconds: Site likelihoods with rate category 16 of 20
     16.87 seconds: Site likelihoods with rate category 18 of 20
     17.01 seconds: Site likelihoods with rate category 20 of 20
Switched to using 20 rate categories (CAT approximation)
Rate categories were divided by 0.909 so that average rate = 1.0
CAT-based log-likelihoods may not be comparable across runs
     17.12 seconds: ML Lengths 1 of 1811 splits
     17.31 seconds: ML Lengths 101 of 1811 splits
     17.50 seconds: ML Lengths 201 of 1811 splits
     17.69 seconds: ML Lengths 301 of 1811 splits
     17.88 seconds: ML Lengths 401 of 1811 splits
     18.07 seconds: ML Lengths 501 of 1811 splits
     18.28 seconds: ML Lengths 601 of 1811 splits
     18.47 seconds: ML Lengths 701 of 1811 splits
     18.67 seconds: ML Lengths

     43.14 seconds: ML Lengths 1701 of 1811 splits
     43.32 seconds: ML Lengths 1801 of 1811 splits
9 rounds ML lengths: LogLk = -447798.199 Max-change 0.0011 Time 43.37
     43.55 seconds: ML Lengths 101 of 1811 splits
     43.72 seconds: ML Lengths 201 of 1811 splits
     43.90 seconds: ML Lengths 301 of 1811 splits
     44.07 seconds: ML Lengths 401 of 1811 splits
     44.25 seconds: ML Lengths 501 of 1811 splits
     44.43 seconds: ML Lengths 601 of 1811 splits
     44.61 seconds: ML Lengths 701 of 1811 splits
     44.79 seconds: ML Lengths 801 of 1811 splits
     44.97 seconds: ML Lengths 901 of 1811 splits
     45.14 seconds: ML Lengths 1001 of 1811 splits
     45.31 seconds: ML Lengths 1101 of 1811 splits
     45.49 seconds: ML Lengths 1201 of 1811 splits
     45.66 seconds: ML Lengths 1301 of 1811 splits
     45.84 seconds: ML Lengths 1401 of 1811 splits
     46.02 seconds: ML Lengths 1501 of 1811 splits
     46.19 seconds: ML Lengths 1601 of 1811 splits
     46.37 seconds: M

# Final step: create the reference package

In [10]:
# run the existing script create_tipp_refpkg.py
db_path = os.path.join(workdir, 'ncbi_taxonomy.db')

cmd = f"python create_tipp_refpkg.py {workdir} {db_path} {refpkg_dir} {num_threads}"
p = Popen(cmd, stdout=PIPE, stderr=STDOUT, text=True, shell=True)
for line in p.stdout:
    print(line.replace('\n', ''))



Building a new DB, current time: 03/05/2025 14:30:03
New DB name:   /home/chengzes/Desktop/Research/softwares/TIPP3/refpkg_scripts/custom_tipp_refpkg/markers-v4/blast/alignment.fasta.db
New DB title:  custom_tipp_refpkg/markers-v4/blast/alignment.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/chengzes/Desktop/Research/softwares/TIPP3/refpkg_scripts/custom_tipp_refpkg/markers-v4/blast/alignment.fasta.db
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 13735 sequences in 0.123385 seconds.


total unique taxids: 10960
Aggregating all sequences to create a BLAST database...
Running makeblastdb...
done creating refpkg: RpsM_COG0099
done creating refpkg: RplA_COG0081
done creating refpkg: RplB_COG0090
done creating refpkg: RplD_COG0088
done creating refpkg: RpsC_COG0092
done computing num sequences: RplA_COG0081
done computing num sequences: RplB_COG0090
done computing num sequences: RplD_COG0088
done computing num sequ